Tutorial 3 - Dynamic Flowsheets
=============================

Introduction
------------

The previous tutorials have looked at developing flowsheets for steady-state processes, however it is often important to study how a process behaves under transient conditions. This tutorial will demonstrate how to create a model for a dynamic process using the IDAES modleing framework.

For this tutorial, we will use a similar process to the previous tutorials, using two CSTRs to perform a simple chemicl reaction. However, for this tutorial we will add a second feed stream and a mixer unit before hte first reactor, to represent the mixing of two reactant streams as shown below.

<img src="mix_2cstrs.png">

<center>ethyl acetate + NaOH $\rightarrow$ sodium acetate + ethanol</center>

In this tutorial, you will learn how to:

* stup a dynamic flowsheet and define the time domain
* add additonal variables and constraints to a Unit Model
* transform a time domain to automatically populate the time domain and define derivative terms
* set initial conditions for a dynamic model
* initialize and solve a dynamic model
* introduce a step change to the model
* plot profiles of key variables as a function of time

Importing Libraries
----------------------------

As before, the first step in creating our model is to import the necessary components from Python, Pyomo and IDAES. FThe componets we will require are discussed below.

Firstly, we need all the components from Pyomo that we have used in the previous tutorials (`ConcreteModel`, `SolverFactory` and  `TransformationFactory` from `pyomo.environ` and `Arc` from `pyomo.network`). In addition to this, we are going to the need the `Constraint` and `Var` components from `pyomo.environ` in order to add additonal variables and constraints to our model.

Import these components below:

In [1]:
from pyomo.environ import (ConcreteModel, SolverFactory, TransformationFactory,
                           Var, Constraint)
from pyomo.network import Arc

Next, we will need the IDAES compoennts we have used in the previous tutorials:

`FlowsheetBlock` from `idaes.core`, 
`idaes.property_models.examples.saponification_thermo`
`idaes.property_models.examples.saponification_reactions`
`CSTR` from `idaes.unit_models`

We will alson need the `Mixer` model from the IDAES unit model library (`idaes.unit_models`)

Import these components below:

In [2]:
from idaes.core import FlowsheetBlock

import idaes.property_models.examples.saponification_thermo as thermo_props
import idaes.property_models.examples.saponification_reactions as \
    reaction_props

from idaes.unit_models import CSTR, Mixer

Finally, we are going to need a tool for plotting our results. For this tutorial, we will use the `pyplot` tool in `matplotlib` (part of most scientific Python distributions), which we will import as `plt` for convenience.

If you do not have `matplotlib` installed in your Python distribution, see you Python package manager for details on how to install it.

The code to import `pyplot` is given below.

In [3]:
import matplotlib.pyplot as plt

Creating Our Model
-----------------------------

Now that we have imported all the tools we will need, we can begin constructing our model. As before, the first step is to create a `ConcreteModel` to hold our flowhseet.

In [4]:
m = ConcreteModel()

Next, we add a `FlowsheetBlock` to our model. In previous tutorials, we have set the `dynamic` argument to be `False`, indicating a steady-state flowsheet. In this case however, we wish to create a dynamic flowsheet, so we set this argumnet to `True`.

Additionally, for a dynamic model, we need to provide some information about the time domain for our model. At a minimum, we need to set the start and end times for our time domain, which for this case will be 0 and 8 seconds respectively. We can also specify specific time points that we wish to ensure are in our time domain, such as points where we know a step-chenge will occur. For this tutorial, we will introduce a step-change at t = 1 second later in the tutorial, so we will include this in our time domain now.

Thus, we want to create a time domain that begins at t = 0, has a point at t = 1 and ends at t = 8. To do this, we use the `time_set` argument when creating our `FlowhseetBlock` and provide these values as a Python list (or equivalent) - i.e. `time_set: [0, 1, 8]` (note that the time vlaues may be integers or floating point numbers at this stage).

The code to do this is shown below:

In [5]:
m.fs = FlowsheetBlock(default={"dynamic": True, "time_set": [0, 1, 8]})

To see what happened in this set, let us `display` the time domain of our flowhseet.

In [6]:
m.fs.time.display()

time : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=Sorted, Bounds=(0.0, 8.0)
    [0.0, 1.0, 8.0]


We should see the following:

```
time : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=Sorted, Bounds=(0.0, 8.0)
    [0.0, 1.0, 8.0]
```

This shows us that our `time` domain is a sorted Pyomo `Set` (`Ordered=Sorted`) with points at 0.0, 1.0 and 8.0 and bounds of 0.0 and 8.0. This is waht we would expect, as we told the `FlowsheetBlock` to create a time domain with these points.

The fact that the `Set` is sorted is important, as indicates the points in the `Set` are in order, allowing us to find the next and previous points if required.

In [ ]:
m.fs.thermo_params = thermo_props.SaponificationParameterBlock()
m.fs.reaction_params = reaction_props.SaponificationReactionParameterBlock(
    default={"property_package": m.fs.thermo_params})

In [ ]:
m.fs.mix = Mixer(default={"dynamic": False,
                          "property_package": m.fs.thermo_params})

In [ ]:
m.fs.Tank1 = CSTR(default={"property_package": m.fs.thermo_params,
                           "reaction_package": m.fs.reaction_params,
                           "has_holdup": True,
                           "has_equilibrium_reactions": False,
                           "has_heat_transfer": True,
                           "has_pressure_change": False,
                           "dynamic": False})
m.fs.Tank2 = CSTR(default={"property_package": m.fs.thermo_params,
                           "reaction_package": m.fs.reaction_params,
                           "has_holdup": True,
                           "has_equilibrium_reactions": False,
                           "has_heat_transfer": True,
                           "has_pressure_change": False})

In [ ]:
m.fs.Tank1.height = Var(m.fs.time,
                        initialize=1.0,
                        doc="Depth of fluid in tank [m]")
m.fs.Tank1.area = Var(initialize=1.0,
                      doc="Cross-sectional area of tank [m^2]")
m.fs.Tank1.flow_coeff = Var(m.fs.time,
                            initialize=5e-5,
                            doc="Tank outlet flow coefficient")

In [ ]:
def geometry(b, t):
    return b.volume[t] == b.area*b.height[t]
m.fs.Tank1.geometry = Constraint(m.fs.time, rule=geometry)

In [ ]:
def outlet_flowrate(b, t):
    return b.control_volume.properties_out[t].flow_vol == b.flow_coeff[t]*b.height[t]
m.fs.Tank1.outlet_flowrate = Constraint(m.fs.time, rule=outlet_flowrate)

In [ ]:
m.fs.Tank2.height = Var(m.fs.time,
                        initialize=1.0,
                        doc="Depth of fluid in tank [m]")
m.fs.Tank2.area = Var(initialize=1.0,
                      doc="Cross-sectional area of tank [m^2]")
m.fs.Tank2.flow_coeff = Var(m.fs.time,
                            initialize=5e-5,
                            doc="Tank outlet flow coefficient")

m.fs.Tank2.geometry = Constraint(m.fs.time, rule=geometry)
m.fs.Tank2.outlet_flowrate = Constraint(m.fs.time, rule=outlet_flowrate)

In [ ]:
m.fs.stream1 = Arc(source=m.fs.mix.outlet, destination=m.fs.Tank1.inlet)
m.fs.stream2 = Arc(source=m.fs.Tank1.outlet, destination=m.fs.Tank2.inlet)

In [ ]:
m.discretizer = TransformationFactory('dae.finite_difference')
m.discretizer.apply_to(m,
                       nfe=200,
                       wrt=m.fs.time,
                       scheme="BACKWARD")

In [ ]:
TransformationFactory("network.expand_arcs").apply_to(m)

In [ ]:
m.fs.mix.inlet_1.flow_vol.fix(0.5)
m.fs.mix.inlet_1.conc_mol_comp[:, "H2O"].fix(55388.0)
m.fs.mix.inlet_1.conc_mol_comp[:, "NaOH"].fix(100.0)
m.fs.mix.inlet_1.conc_mol_comp[:, "EthylAcetate"].fix(0.0)
m.fs.mix.inlet_1.conc_mol_comp[:, "SodiumAcetate"].fix(0.0)
m.fs.mix.inlet_1.conc_mol_comp[:, "Ethanol"].fix(0.0)
m.fs.mix.inlet_1.temperature.fix(303.15)
m.fs.mix.inlet_1.pressure.fix(101325.0)

m.fs.mix.inlet_2.flow_vol.fix(0.5)
m.fs.mix.inlet_2.conc_mol_comp[:, "H2O"].fix(55388.0)
m.fs.mix.inlet_2.conc_mol_comp[:, "NaOH"].fix(0.0)
m.fs.mix.inlet_2.conc_mol_comp[:, "EthylAcetate"].fix(100.0)
m.fs.mix.inlet_2.conc_mol_comp[:, "SodiumAcetate"].fix(0.0)
m.fs.mix.inlet_2.conc_mol_comp[:, "Ethanol"].fix(0.0)
m.fs.mix.inlet_2.temperature.fix(303.15)
m.fs.mix.inlet_2.pressure.fix(101325.0)

In [ ]:
m.fs.Tank1.area.fix(1.0)
m.fs.Tank1.flow_coeff.fix(0.5)
m.fs.Tank1.heat_duty.fix(0.0)

m.fs.Tank2.area.fix(1.0)
m.fs.Tank2.flow_coeff.fix(0.5)
m.fs.Tank2.heat_duty.fix(0.0)

In [ ]:
m.fs.fix_initial_conditions()

In [ ]:
m.fs.mix.initialize()

In [ ]:
m.fs.Tank1.initialize(state_args={
            "flow_vol": m.fs.mix.outlet.flow_vol[0].value,
            "conc_mol_comp": {"H2O": m.fs.mix.outlet.conc_mol_comp[0, "H2O"].value,
                              "NaOH": m.fs.mix.outlet.conc_mol_comp[0, "NaOH"].value,
                              "EthylAcetate": m.fs.mix.outlet.conc_mol_comp[0, "EthylAcetate"].value,
                              "SodiumAcetate": m.fs.mix.outlet.conc_mol_comp[0, "SodiumAcetate"].value,
                              "Ethanol": m.fs.mix.outlet.conc_mol_comp[0, "Ethanol"].value},
            "temperature": m.fs.mix.outlet.temperature[0].value,
            "pressure": m.fs.mix.outlet.pressure[0].value})

In [ ]:
m.fs.Tank2.initialize(state_args={
            "flow_vol": m.fs.Tank1.outlet.flow_vol[0].value,
            "conc_mol_comp": {"H2O": m.fs.Tank1.outlet.conc_mol_comp[0, "H2O"].value,
                              "NaOH": m.fs.Tank1.outlet.conc_mol_comp[0, "NaOH"].value,
                              "EthylAcetate": m.fs.Tank1.outlet.conc_mol_comp[0, "EthylAcetate"].value,
                              "SodiumAcetate": m.fs.Tank1.outlet.conc_mol_comp[0, "SodiumAcetate"].value,
                              "Ethanol": m.fs.Tank1.outlet.conc_mol_comp[0, "Ethanol"].value},
            "temperature": m.fs.Tank1.outlet.temperature[0].value,
            "pressure": m.fs.Tank1.outlet.pressure[0].value})

In [ ]:
solver = SolverFactory('ipopt')
results = solver.solve(m.fs)

In [ ]:
for t in m.fs.time:
    if t > 1.0:
        m.fs.mix.inlet_2.conc_mol_comp[t, "EthylAcetate"].fix(90.0)
results = solver.solve(m.fs)

In [ ]:
print(results)

In [ ]:
plt.figure("Tank 1 Outlet")
plt.plot(m.fs.time,
         list(m.fs.Tank1.outlet.conc_mol_comp[:, "NaOH"].value),
         label='NaOH')
plt.plot(m.fs.time,
         list(m.fs.Tank1.outlet.conc_mol_comp[:, "EthylAcetate"].value),
         label='EthylAcetate')
plt.plot(m.fs.time,
         list(m.fs.Tank1.outlet.conc_mol_comp[:, "SodiumAcetate"].value),
         label='SodiumAcetate')
plt.plot(m.fs.time,
         list(m.fs.Tank1.outlet.conc_mol_comp[:, "NaOH"].value),
         label='Ethanol')
plt.legend()
plt.grid()
plt.xlabel("Time [s]")
plt.ylabel("Concentration [mol/m^3]")

In [ ]:
plt.figure("Tank 2 Outlet")
plt.plot(m.fs.time,
         list(m.fs.Tank2.outlet.conc_mol_comp[:, "NaOH"].value),
         label='NaOH')
plt.plot(m.fs.time,
         list(m.fs.Tank2.outlet.conc_mol_comp[:, "EthylAcetate"].value),
         label='EthylAcetate')
plt.plot(m.fs.time,
         list(m.fs.Tank2.outlet.conc_mol_comp[:, "SodiumAcetate"].value),
         label='SodiumAcetate')
plt.plot(m.fs.time,
         list(m.fs.Tank2.outlet.conc_mol_comp[:, "NaOH"].value),
         label='Ethanol')
plt.legend()
plt.grid()
plt.xlabel("Time [s]")
plt.ylabel("Concentration [mol/m^3]")